In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
import ta  # Technical Analysis library

# Carga de datos
data = pd.read_csv('./BitcoinHistoricalData.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data[['Close']]

# Feature Engineering: Añadir indicadores técnicos
data['SMA'] = ta.trend.sma_indicator(data['Close'], window=20)
data['RSI'] = ta.momentum.rsi(data['Close'], window=14)
data['MACD'] = ta.trend.macd(data['Close'])

# Preprocesamiento de datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.dropna())

# Crear etiquetas
data['Change'] = data['Close'].pct_change()
data['Label'] = data['Change'].apply(lambda x: 0 if x < -0.01 else (2 if x > 0.01 else 1))
data.dropna(inplace=True)

# Preprocesamiento de etiquetas
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])
labels = to_categorical(data['Label'])

# Preparación de secuencias
def create_sequences(data, labels, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(labels[i + seq_length])
    return np.array(X), np.array(y)

SEQ_LENGTH = 60
X, y = create_sequences(data_scaled, labels, SEQ_LENGTH)

# Dividir en conjuntos de entrenamiento y prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Construcción del modelo
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(3, activation='softmax'))  # Cambiar a 3 neuronas para 3 clases

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento
model.fit(X_train, y_train, batch_size=32, epochs=50)

# Predicciones
predictions = model.predict(X_test)

# Convertir predicciones y etiquetas a etiquetas originales
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

# Reporte de clasificación y matriz de confusión
print(classification_report(true_labels, predicted_labels, target_names=['Bajar', 'Mantener', 'Subir']))
print(confusion_matrix(true_labels, predicted_labels))

# Visualización
graph = pd.DataFrame({'Date': data.index[split + SEQ_LENGTH:], 'Real': true_labels, 'Prediccion': predicted_labels})
plt.figure(figsize=(20, 10))
sns.lineplot(x='Date', y='Real', data=graph, label='Real')
sns.lineplot(x='Date', y='Prediccion', data=graph, label='Prediccion')
plt.title('Bitcoin')
plt.xlabel('Fecha')
plt.ylabel('Etiqueta')
plt.legend()
plt.show()


KeyError: "None of [Index(['Close'], dtype='object')] are in the [columns]"